In [1]:
import torch
import numpy as np
from train import create_dataloader
from model import make_model

SyntaxError: invalid syntax (1417961504.py, line 3)

In [ ]:
config = {'dataset': 'scada1_1', 'l_win': 500, 'pre_mask': 200, 'post_mask': 400,
          'batch_size': 32, 'shuffle': True, 'dataloader_num_workers': 4, 'num_epoch': 50, 'model_path': '../models/'}

In [ ]:
model = make_model(N=6, d_model=16, l_win=config['l_win'], d_ff=128, h=1, dropout=0.1).float()
model.load_state_dict(torch.load(config['model_path'] + "best_train_49.pth"))
model.eval()

In [ ]:
class EvalDataset(Dataset):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.load_dataset(self.config['dataset'])
        self.input_mask = self._create_window_mask()

    def __len__(self):
        return self.rolling_windows.shape[0]

    def __getitem__(self, index):
        input = self.rolling_windows[index, :, :]
        sample = {"input": input, "target": input}
        return sample

    def load_dataset(self, dataset):
        data_dir = '../data/scada/'
        data = np.load(data_dir + dataset + '.npz')

        # slice training set into rolling windows
        self.rolling_windows = np.lib.stride_tricks.sliding_window_view(
            data['test'], self.config['l_win'], axis=0, writeable=True).transpose(0, 2, 1)

    def _create_window_mask(self):
        """Mask out the defined positions."""
        config = self.config
        attn_shape = (1, config['l_win'], self.rolling_windows.shape[-1])
        window_mask = np.ones(attn_shape).astype('float')
        window_mask[:, config['pre_mask']:config['post_mask'], :] = 0.0
        return torch.from_numpy(window_mask)

    def mask_input(self):
        self.input_mask = self.input_mask.masked_fill(self.input_mask == 0, float(-1e9)).masked_fill(self.input_mask == 1, 0)
        self.rolling_windows = np.add(self.rolling_windows, self.input_mask)


In [ ]:
data_dir = '../data/scada/'
data = np.load(data_dir + config['dataset'] + '.npz')
print(data['test'].shape)
data.files

In [ ]:
data['training'].shape

In [ ]:
data['idx_anomaly_test'].shape

In [ ]:
rolling_windows = np.lib.stride_tricks.sliding_window_view(
            data['test'], config['l_win'], axis=0, writeable=True).transpose(0, 2, 1)

In [ ]:
dataset = EvalDataset(config)

In [ ]:
data_loader = create_dataloader(dataset, config)

In [ ]:
for i, batch in enumerate(data_loader):
    out = model(, src_mask=None)
    bre